In [1]:
import pandas
import json
from tqdm import tqdm
import re
import os
import requests
import datetime
from bs4 import BeautifulSoup as bs
from collections import defaultdict

In [31]:
def printer(data):
    """
    get: list of dictionaries
    print: 'title' and 'link' from each dictionari
    """
    for e in data:
        print(e['title'])
        print(e['href'])
        
        
def finder(data, select):
    """
    get: list of dictionaries
    return: list of dictionaries minus sublist of dictionaries by selector,
            sublist of dictionaries by selector
    """
    run_date = data[0]
    g = [run_date]
    v = [run_date]
    
    for e in data[1:]:
        t = e['title']
        if (re.match(r'[ДПА][а-яВ\s]+\d{,2}[\.\s][\dа-я]+[\.\s]\d{,4}', t)
            and select in t):
            g.append(e)
        else:
            v.append(e)
    return v, g


def xa0(string):
    """
    Hey, mister \xa0, GTFOH! 
    """
    if '\xa0' in string:
        string = string.replace('\xa0', '')
    return string


def check_keys(dictofdicts):
    """
    get: dictionari of dictionaries
    print: keys of dictionaries
    """
    q = []
    for k, v in w.items():
        for i, l in v.items():
            q.append(i)
    print(set(q))
    
    
def get_date(yyyy, mm, dd):
    """
    get: mess
    return: DateTime format
    """
    mm_eng = {'Янв':'Jan', 'Фев':'Feb', 'Мар':'Mar', 'Апр':'Apr', 
              'Мая':'May', 'Июн':'Jun', 'Июл':'Jul', 'Авг':'Aug', 
              'Сен':'Sep', 'Окт':'Oct', 'Ноя':'Nov', 'Дек':'Dec'}
    
    g = dd+'/'+mm_eng[mm.capitalize()]+'/'+yyyy
    d = datetime.datetime.strptime(g, '%d/%b/%Y').date()
    return d


def DieWalkuere():
    """
    return: json from career.hse.ru/news
    """
    run_datetime = datetime.datetime.now().strftime("%Y-%m-%d")
    m_r = requests.get('https://career.hse.ru/news/') # делаем запрос к сайту
    soup = bs(m_r.text, 'lxml')
    num_of_cycles = int(soup.find_all('a', {'class' : 'pages__page'})[-1].text) # узнаём количество страниц в новостной ленте
    hse_career_posts = [run_datetime] # список в который будут записаны результаты

    with tqdm(total=num_of_cycles) as pbar:
        for i in range(num_of_cycles): # создаём цикл, с количеством итераций равным количеству страниц в новостной ленте
            r = requests.get(f'https://career.hse.ru/news/page{i}.html') # запрошиваем нужную страницу
            soup = bs(r.text, 'lxml') # применяем к ее html BeautifulSoup, преобразуем в lxml

            posts_content = soup.find_all("div", {"class": "post__content"}) # вычленяем все div'ы в которых содежраться новостные посты
            posts_meta = soup.find_all("div", {"class": "post-meta__date"}) # вычленяем div'ы с мета-данными постов

            qq = list(zip(posts_content, posts_meta)) # объеденяем списоки div'ов с постами и div'ов с мета-данными в список кортежей

            for e in qq:
                title = e[0].find('a').text # заголовок поста
                href = e[0].find('a')['href'] # ссылка на страницу с конкретной новостью
                text_content = e[0].find('div', {'class' : 'post__text'}).text # лид из поста

                # из элемента с мета-данными получаем год, месяц и день, записанные в очень своеобразном формате
                dd = e[1].find('div', {'class' : 'post-meta__day'}).text 
                mm = e[1].find('div', {'class' : 'post-meta__month'}).text
                yyyy = e[1].find('div', {'class' : 'post-meta__year'}).text

                d = str(get_date(yyyy, mm, dd)) # преобразуем дату в стандарт iso

                e_dict = {'title':title, 'href':href, 
                          'text_content':text_content, 'date':d} # записываем все полученыеданне в словарь
                hse_career_posts.append(e_dict) # помещаем словарь в список
            pbar.update()
            
    with tqdm(total=len(hse_career_posts[1:])) as pbar:
        for e in hse_career_posts[1:]:
            try:
                url = e['href']
                if 'http:' in url:
                    url = url.replace('http:', 'https:')
                if not 'https:' in url:
                    url = 'https:'+url
                r = requests.get(url).text
                e['href_content_html'] = r
            except:
                e['href_content_html'] = ''
                pass
            pbar.update()
            
    return hse_career_posts


def Fricka(listofobj):
    """
    get: list of dictionaries
    return: list of strings + run_date
    """
    title_p = "margin-bottom: .0001pt; text-align: center; text-indent: 1.0cm; line-height: 150%;"
    Liste = [listofobj[0]]
    with tqdm(total=len(listofobj[1:])) as pbar:
        for obj in listofobj[1:]:
            href = obj.get('href_content_html') # html код страницы с дайджестом
            o = bs(href, 'lxml') # применяем BeautifulSoup, преобразуем в lxml
            core_div = o.find('div', {'class': 'post__text'}) 
            list_of_p = core_div.find_all('p') # вычленяем блоки с текстами вакансий
            for p in list_of_p:
                if p.get('style') == title_p: # находим блоки в которых заголовки вакансий
                    tex = p.text
                    tex = xa0(tex) #удаляем неразрывные пробелы
                    if tex != '':
                        Liste.append('o'+tex) # добавляем в начало символьный маркер для заголовка вакансии
                        Liste.append(obj['date']) # добавляем текст блока в список
                else:
                    spans = p.find_all('span') # находим в блоке все span'ы
                    for s in spans:
                        tex = s.text # достаём из каждого текст
                        tex = xa0(tex) # удаляем неразрывные пробелы
                        if tex != '':
                            Liste.append(tex) # добавляем в список
            pbar.update()
    return Liste


def Wotan(listofstr):
    """
    get: list of strings
    return: list: run_date and dictionari of dictionaries
            example: {title_1 : {aspect_1 : text_1, aspect_2 : text_2}, 
                      title_2 : {aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      }
    """
    
    keys = ({'Контакты:':'contacts', 'О компании:':'about_company', 
             'Обязанности:':'responsibilities', 'Условия:':'conditions', 
             'Окомпании:':'about_company', 'Требования:':'demands'}) # названия подзаголовков
    WW = [listofstr[0]]
    Woerterbuch = defaultdict(dict) # структура словарь словарей
    current_date = ''
    current_title = ''
    curretn_aspect = ''
    
    with tqdm(total=len(listofstr[1:])) as pbar:
        # заголовок вакансии – ключ, его значение – словарь, ключи в котором аспекты каждой вакансии
        for i, e in enumerate(listofstr[1:]):
            t = re.compile(r'[o]\d+[\.][\s]\b') # сохраняем паттерн для вычленения заголовкой
            if t.match(e): # распознаём заголовок
                nomore = t.match(e).group() 
                current_title = e.replace(nomore, '')
                Woerterbuch[current_title]
            try:   
                if re.match(r'\d{4}[-]\d{2}[-]\d{2}\b', e): # распознаем дату
                    current_date = e
                    Woerterbuch[current_title]['date'] = current_date
                elif e.strip() in keys: # распознаем аспекты вакании
                    curretn_aspect = keys[e.strip()]
                    Woerterbuch[current_title][curretn_aspect] = ''
                else: # всё остальное – содержание аспектов
                    Woerterbuch[current_title][curretn_aspect]+=e
            except:
                pass
            pbar.update()
        WW.append(Woerterbuch)
    return WW


def Bruennhilde(dictofdicts):
    """
    get: dictionary of dictionaries
            example: {title_1 : {aspect_1 : text_1, aspect_2 : text_2}, 
                      title_2 : {aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      }
    return: list of dictionaries
            example: [{title : title_1, aspect_1 : text_1, aspect_2 : text_2}, 
                      {title : title_2, aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      ]
    """
    with open('metro.txt', 'r', encoding='utf-8') as f:
        ms = f.read().split('\n')
    
    Siegfried = []
    # меняем структуру данных для простоты табличного представления, и достаём дополнительные данные
    for k, v in dictofdicts.items():
        Fafnir = {}
        c = k.split(' в ')
        n = re.compile(r'[А-ЯЁA-Z][\w\s]+')
        # вычленяем название компании из заговоловка вакансии
        if len(c) > 1 and n.search(c[-1]):
            Fafnir['title'] = k
            Fafnir['company'] = n.search(c[-1]).group()
        else:
            Fafnir['title'] = k
        money = re.compile(r'[\d][\d\s]+[руб]{1,3}[.]{,1}\s')
        metrore = re.compile(r'\s[м][.][\s][А-ЯЁ][а-я]+\s')
        # пытаемся найти информацию о деньгах и локациях (относительно метро)
        if 'conditions' in v:
            cond = v['conditions']
            if money.search(cond):
                Fafnir['money'] = ', '.join(money.findall(cond))
            metro = []
            if metrore.search(cond):
                for e in ms:
                    if e in cond:
                        metro.append('м. ' + e)
                if len(metro) > 0:
                    Fafnir['metro'] = ', '.join(metro)
            Fafnir.update(v)
        if len(Fafnir) > 2:
            Siegfried.append(Fafnir)
    return Siegfried

In [17]:
#This cell downloads the website, be careful!

Data = DieWalkuere()

with open('hse_career_department_website_posts.json', 'w') as f:
        json.dump(Data, f)

100%|██████████| 459/459 [07:11<00:00,  1.81s/it]


In [18]:
with open('hse_career_department_website_posts.json', 'r') as f:
        Data = json.load(f)

In [32]:
# данные о вакансиях для студентов

data2, stud = finder(Data, 'студент')


step1_stud = Fricka(stud)
step2_stud = Wotan(step1_stud)
step3_stud = Bruennhilde(step2_stud[1])

Table_stud = pandas.DataFrame(step3_stud)

100%|██████████| 30918/30918 [00:00<00:00, 143512.04it/s]


In [33]:
# данные о вакансиях для выпускников

data3, grad = finder(data2, 'выпускник')

step1_grad = Fricka(grad)
step2_grad = Wotan(step1_grad)
step3_grad = Bruennhilde(step2_grad[1])

Table_grad = pandas.DataFrame(step3_grad)

100%|██████████| 23908/23908 [00:00<00:00, 138908.40it/s]


In [35]:
#Table_stud

In [ ]:
#Table_grad

In [37]:
# сохраня

rdd = 'results'+step2_stud[0]

os.mkdir(rdd)

with open(rdd+'\Table_stud'+step2_stud[0]+'.json', 'w') as f:
        json.dump(step3_stud, f)
Table_stud.to_csv(rdd+'\Table_stud'+step2_stud[0]+'.csv', encoding='utf-8')


with open(rdd+'\Table_grad'+step2_grad[0]+'.json', 'w') as f:
        json.dump(step3_grad, f)
Table_grad.to_csv(rdd+'\Table_grad'+step2_grad[0]+'.csv', encoding='utf-8')


with open(rdd+'\ostatki'+step2_grad[0]+'.json', 'w') as f:
        json.dump(data3, f)